# Gry o sumie zerowej: twierdzenie o minimaksie i programowanie liniowe

Jako zastosowanie silnego twierdzenia o dualności udowodnimy następujące twierdzenie o minimaksie dla gier o sumie zerowej.

**Twierdzenie o minimaksie dla gier o sumie zerowej**.
*Dla każdej gry o sumie zerowej istnieje równowaga Nasha a
optymalne strategie obu graczy można efektywnie wyznaczyć za pomocą programowania liniowego.*

Przed dowodem twierdzenia wyjaśnię użyte w nim sformułowania. Zaczniemy od analizy przykładu.

# Gra "Pułkownik Blotto"

Pułkownik Blotto przygotowuje się do bitwy o trzy górskie przełęcze. Dysponuje - tak jak jego przeciwnik - pięcioma regimentami. Ten, kto wyśle więcej regimentów na przełęcz zajmie ją; jeżeli obaj poślą na przełęcz tyle samo regimentów, nie zajmie jej nikt. Ten, który zajmie więcej przełęczy wygrywa bitwę; jeżeli zajmą tyle samo bitwa jest nierozstrzygnięta (kończy się remisem).

Jako, że żadna z przełęczy nie jest strategicznie ważniejsza, strategia pułkownika Blotto (i jego przeciwnika) polega na podzieleniu swoich regimentów na trzy grupy a następnie posłanie tych grup na losowo wybrane przełęcze. Obaj chcą zmaksymalizować *swoje* prawdopodobieństwa zwycięstwa (które ze względu na remisy nie muszą się sumować do jedynki). W jaki sposób powinni podzielić swoje regimenty?

### Macierz wypłat

Gra pułkownik Blotto to przykład skończonej gry dwuosobowej o sumie zerowej. W grze takiej każdy z dwóch graczy (jednocześnie) wybiera ruch ze skończonego zbioru możliwości (w naszym przypadku - spośród możliwych podziałów regimentów). Następnie każdy z graczy otrzymuje *wypłatę* a wypłaty te sumują się do zera. W naszym przypadku wypłatą pułkownika będzie +1 w przypadku wygranej pułkownika, 0 w przypadku remisu i -1 w przypadku wygranej przeciwnika. Są to wypłaty z punktu widzenia pułkownika Blotto: wypłaty jego przeciwnika to -1, 0 i +1 odpowiednio w każdym z tych przypadków.

Skoro regimenty posyłane są na przełęcze losowo, podział (5, 0, 0) czy (0, 5, 0) to ten sam podział. Dlatego zbiór możliwych ruchów możemy zapisać jako (0, 0, 5), (0, 1, 4), (0, 2, 3), (1, 1, 3), (1, 2, 2) (liczby regimentów zapisując w kolejności niemalejącej). Reguły gry możemy zapisać za pomocą następującej *macierzy wypłat*.

$$
\begin{array}{c|ccccc}
 & (0, 0, 5) & (0, 1, 4) & (0, 2, 3) & (1, 1, 3) & (1, 2, 2) \\
 \hline
 (0, 0, 5) & 0 & -\frac 13 & - \frac 13 & -1 & -1 \\
 (0, 1, 4) & \frac 13 & 0 & 0 & -\frac 13 & -\frac 23 \\
 (0, 2, 3) & \frac 13 & 0 & 0 & 0 & \frac 13 \\
 (1, 1, 3) & 1 & \frac 13 & 0 & 0 & -\frac 13 \\
 (1, 2, 2) & 1 & \frac 23 & -\frac 13 & \frac 13 & 0 \\
\end{array}
$$

W kolumnach zapisane są ruchy przeciwnika. W wiersza zapisane są ruchy pułkownika Blotto. Na przecięciu wiersza i kolumny wpisana jest wartość oczekiwana wypłaty jeżeli gracze wybiorą dane ruchy. Przykładowo, wartość $-\frac 13$ w drugiej kolumnie pierwszego wiersza odpowiada wyborowi $(0, 0, 5)$ pułkownika i $(0, 1, 4)$ jego przeciwnika. Wartość oczekiwana wypłaty to

$$
  \frac 23 \cdot 0 + \frac 13 \cdot -1 = -\frac 13.
$$

Rzeczywiście, z prawdopodobieństwem $\frac 23$ oddział pułkownika Blotto złożony z pięciu regimentów spotka się z oddziałem przeciwnika złożonym z jednego lub czterech regimentów. Wtedy obaj zajmą po jednej przełęczy i bitwa zakończy się remisem. Gdy jednak oddział pięciu regimentów trafi na przełęcz, na którą przeciwnik nie posłał żadnego regimentu, to przeciwnik zajmie dwie pozostałe przełęcze i wygra bitwę. Stąd podana oczekiwana wartość ruchu, zapisana z punktu widzenia pułkownika Blotto, który gra wierszami.


In [1]:
# 0 0 5 - 0 1 4 - remis
# 0 0 5 - 0 4 1 - remis
# 0 0 5 - 1 0 4 - remis
# 0 0 5 - 4 0 1 - remis
# 0 0 5 - 1 4 0 - wygrana przeciwnika: -1
# 0 0 5 - 4 1 0 - wygrana przeciwnika: -1

ruchy = []
for i in range(6):
    for j in range(6 - i):
        m = 5 - i - j
        ruchy.append((i,j,m))
print(len(ruchy))

21


### Strategia optymalna - wersja pesymistyczna

Nie znając strategii przeciwnika pułkownik Blotto może zdecydować się na wybór strategii (ruchu) dającej największą oczekiwaną wypłatę *w najgorszym przypadku*. Jedynym kandydatem na taką strategię jest ruch $(0, 2, 3)$, gwarantujący wypłatę co najmniej $0$. Jeżeli pułkownik Blotto wybierze jakikolwiek inny ruch, to ryzykuje stratę: np. na ruch $(0, 1, 4)$ przeciwnik może odpowiedzieć $(1, 2, 2)$, z oczekiwaną wypłątą pułkownika równą $-\frac 23$.

Naturalnie przeciwnik nie zna strategii pułkownika Blotto i może postąpić tak samo: wybrać ruch $(0, 2, 3)$ dający mu wypłątę co najmniej $0$.

Jeżeli obaj wybiorą takie strategie, to spełnią się ich pesymistyczne scenariusze: obaj dostaną średnią wypłatę $0$. Jednak żaden z nich nie będzie miał powodu do żalu: jeżeli wybrałby inną strategię (jednostronnie) to jego maksymalna wypłata nie byłaby wyższa, a mógłby na tym stracić.

Taki wybór strategii już za chwilę nazwiemy *równowagą Nasha* po wprowadzeniu bardziej ogólnej definicji strategii.

CoCalc## Gra "Papier Kamień Nożyczki" (RoShamBo)

Alicja i Bob jednocześnie pokazują jeden z trzech gestów ręką oznaczających kamień (**R**ock), papier (**P**aper) lub nożyczki (**S**cissors). Jeżeli oboje pokażą ten sam gest, gra kończy się remisem. W przeciwnym przypadku reguły są cykliczne: nożyczki "tną" papier; papier "przykrywa" kamień; kamień "rozbija" nożyczki. Reguły podsumowuje następująca macierz wypłat.

$$
\begin{array}{c|ccc}
 & R & P & S \\
\hline
R & 0 & -1 & 1 \\
P & 1 & 0 & -1 \\
S & -1 & 1 & 0
\end{array}
$$

Ruchy Alicji wpisane są w wierszach ("Alicja gra wierszami"), ruchy Boba wpisane są w kolumnach. Wypłata podana jest z punktu widzenia Alicji: w trzecim wierszu pierwszej kolumnie jest liczba $-1$ oznaczająca, że jeżeli Alicja zagra nożyczki a Bob kamień, to Alicja straci jeden punkt.

W tej grze nie ma równowagi takiej, jak w poprzednim przykładzie: nie ma takiego ch wyboru ruchów obu graczy, przy którym których żadnemu z graczy nie opłaca się swojego wyboru zmienić. Ale to dlatego, że przyjęcie jako strategii twardego pokazywania "kamień" w każdej rundzie nie jest rozsądną strategią. Lepiej zmieniać swoje wybory. Jeżeli Alicja będzie losowała swój gest z równym prawdopodobieństwem ($\frac 13$ kamień, $\frac 13$ papier i $\frac 13$ nożyczki) to wartością oczekiwaną (niezależnie od strategii Boba!) wypłaty będzie

$$
  \frac 13 \cdot 0 + \frac 13 \cdot 1 + \frac 13 \cdot (-1) = 0.
$$

Jeżeli Bob wybierze tę samą strategię, to żadnemu z graczy nie będzie się opłacało zmienić strategii - bo i tak wyjdzie na zero. (Proszę zwrócić uwagę, że gracz zawsze pokazujący kamień też zremisuje na gracza grającego losowo - dlatego [jeżeli w puli graczy nie wszyscy grają optymalnie to czasem warto odstąpić od strategii losowej](https://webdocs.cs.ualberta.ca/~darse/rsbpc.html).)

## Strategia mieszana

Odejdźmy od przykładów i rozważmy przypadek ogólny. Mamy dwóch graczy (Alicję i Boba). Alicja może wykonać jeden z $m$ dostępnych ruchów (ma do wyboru $m$ **czystych** strategii); Bob może wykonać jeden z $n$ dostępnych ruchów (gra nie musi być symetryczna). Znamy też macierz wypłat $M$ o $m$ wierszach i $n$ kolumnach (Alicja "gra wierszami"). Współczynnik $m_{ij}$ tej macierzy to wypłata Alicji jeżeli zagra on $i$-ty ruch a Bob zagra $j$-ty ruch. Jeżeli $m_{ij}$ jest ujemne, to "wypłata" ta oznacza przegraną Alicji.

**Definicja**. **Strategia mieszana** (nazywana odtąd po prostu **strategią**) gracza to rozkład prawdopodobieństwa z jakim wykonuje on dostępne ruchy. 


Strategię Alicji będziemy kodować za pomocą $m$-wymiarowego wektora

$$
  \bm{x} = (x_1, x_2, \ldots, x_m), \sum_{i = 1}^m x_i = 1, \bm{x} \geq \bm{0}.
$$

(Pogrubiona czcionka oznacza wielkości wektorowe.) Strategię Boba będziemy kodować za pomocą $n$-wymiarowego wektora

$$
  \bm{y} = (y_1, y_2, \ldots, y_m), \sum_{i = 1}^m y_i = 1, \bm{y} \geq \bm{0}.
$$

**Definicja**. Dana jest strategia $\bm{x}$ Alicji i $\bm{y}$ Boba. **Oczekiwana wypłata** (z punktu widzenia Alicji) gdy gra strategią $\bm{x}$ przeciw strategii $\bm{y}$ to

$$
\sum_{i,j} m_{ij} P(\text{Alicja zagra } i \text{ a Bob zagra } j) =
$$
$$
  \sum_{i, j} m_{ij} P_{\bm{x}}(\text{Alicja zagra } i) \cdot P_{\bm{y}}(\text{Bob zagra } j) =
$$
$$
  \sum_{i, j} m_{ij} x_i y_j =
$$
$$
  \bm{x}^T M \bm{y}.
$$

Założona jest niezależność zdarzeń: Alicja i Bob wybierają swoje ruchy jednocześnie, nie znając wyboru drugiego gracza. Sformalizujemy teraz zasadę pułkownika Blotto: szykuj się na najgorsze. Alicja wybierając swoją strategię $\bm{x}$ będzie zakładać, że Bob zagra (lub po prostu, że *może* zagrać) najlepszą możliwową odpowiedź: strategię $\bm{y}$ minimalizującą jej oczekiwaną wypłatę $\bm{x}^T M \bm{y}$. Tak samo Bob oczekuje, że Alicja wybierze strategię maksymalizującą $\bm{x}^T M \bm{y}$.

Oczekiwania te oddają funkcje $\beta$ i $\alpha$ zdefiniowane następująco

$$
\beta(\bm{x}) = \min_\bm{y} \bm{x}^T M \bm{y}, \quad\quad\quad \alpha(\bm{y}) = \max_\bm{x} \bm{x}^T M \bm{y}.
$$

Liczba $\beta(\bm{x})$ to najlepsza (czyli najmniejsza - strategię dobiera Bob) wypłata jaką może uzyskać Alicja grając strategię $\bm{x}$ przeciwko Bobowi. Podobnie $\alpha(\bm{y})$ to jest najlepsza (największa) wypłata, jaką może uzyskać Alicja grając na strategię $\bm{y}$ Boba.

Strategia $\bm{y}_0$ jest najlepszą odpowiedzią Boba na strategię $\bm{x}$ Alicji jedynie jeśli $\bm{x}^T M \bm{y}_0 = \beta{x}$.

Zwróćmy uwagę, że $\alpha$ i $\beta$ to dobrze określone funkcje, bo maksimum i minimum liczymy na zbiorze zwartym a optymalizowana funkcja jest ciągła.

## Równowaga Nasha

**Definicja**. Para $(\tilde\bm{x}, \tilde\bm{y})$ strategii jest **równowagą Nasha** jeżeli $\tilde\bm{x}$ jest najlepszą odpowiedzią na $\tilde\bm{y}$ i $\tilde\bm{y}$ jest najlepszą odpowiedzią na $\tilde\bm{x}$. Rownoważnie, para ta jest równowagą Nasha jeżeli
$$
  \beta(\tilde\bm{x}) = \tilde\bm{x}^T M \tilde\bm{y} = \alpha(\tilde\bm{y}).
$$

Powiemy, że strategia $\tilde\bm{x}$ Alicji jest **optymalna w najgorszym przypadku** lub **pesymistycznie optymalna** jeżeli $$\beta(\tilde\bm{x}) = \max_\bm{x} \beta(\bm{x})$$ Alicja oczekuje, że Bob zawsze zagra najlepszą odpowiedź na każdą z jej strategii i wybiera strategię maksymalizującą jej wypłatę w tym przypadku. Podobnie, strategia $\tilde\bm{y}$ Boba jest pesymistycznie optymalna jeżeli $$\alpha(\tilde\bm{y}) = \min_{\bm{y}} \alpha(\bm{y})$$.

**Lemat**.

1. Mamy
$$
\max_\bm{x} \beta(\bm{x}) \leq \min_\bm{y} \alpha(\bm{y}).
$$
Dokładniej, dla dowolnych dwóch strategii $\bm{x}$, $\bm{y}$ mamy
$$
\beta(\bm{x}) \leq \bm{x}^T M \bm{y} \leq \alpha(\bm{y}).
$$

2. Jeżeli para strategii $(\tilde\bm{x}, \tilde\bm{y})$ jest równowagą Nasha, to zarówno $\tilde\bm{x}$ jak i $\tilde\bm{y}$ są pesymistycznie optymalne.

3. Jeżeli strategie $\bm{x}$ i $\bm{y}$ spełniają $\beta(\bm{x}) = \alpha(\bm{y})$, to tworzą one równowagę Nasha.

**Dowód**. Pierwsze zdanie pierwszego punktu wynika z drugiego zdania, które jest bezpośrednią konsekwencją definicji.

Aby udowodnić 2. zauważmy, że dla dowolnego $\bm{x}$ mamy $\beta(\bm{x}) \leq \alpha(\tilde\bm{y})$ z 1. więc jeżeli $\beta(\tilde\bm{x}) = \alpha(\tilde\bm{y})$, 
to $\beta(\bm{x}) \leq \beta(\tilde\bm{x})$. Więc $\tilde\bm{x}$ jest pesymistycznie optymalna. Analogicznie $\tilde\bm{y}$, co kończy dowód 2.

Aby udowodnić 3. zauważmy, że jeżeli $\beta(\tilde\bm{x}) = \alpha(\tilde\bm{y})$, to z 1. mamy $\beta(\tilde\bm{x}) = \tilde\bm{x}^T M \tilde\bm{y} = \alpha(\tilde\bm{y})$, czyli $(\tilde\bm{x}, \tilde\bm{y})$ tworzą równowagę Nasha.


## Twierdzenie o minimaksie

**Twierdzenie o minimaksie**. W każdej grze o sumie zerowej, strategie pesymistycznie optymalne dla obu graczy istnieją i mogą być efektywnie wyznaczone metodami programowania liniowego. Jeżeli $\tilde\bm{x}$ jest strategią pesymistycznie optymalną Alicji i $\tilde\bm{y}$ jest strategią pesymistycnie optymalną Boba, to $(\tilde\bm{x}, \tilde\bm{y})$ tworzy równowagę Nasha i liczba $\beta(\tilde\bm{x}) = \tilde\bm{x}^T M \tilde\bm{y} = \alpha(\tilde\bm{y})$ jest taka sama dla wszystkich pesymistycznie optymalnych strategii $\tilde\bm{x}$, $\tilde\bm{y}$.

Wartość $\tilde\bm{x}^T M \tilde\bm{y}$, czyli oczekiwaną wartość gry w równowadze Nasha, nazywamy **wartością gry**. Z poprzedniego lematu wiemy, że $(\tilde\bm{x}, \tilde\bm{y})$ tworzą równowagę Nasha jedynie jeśli są jednocześnie pesymistycznie optymalne.

Twierdzenie o minimaksie pokazuje, że strategia "pesymistyczna" jest optymalna w tym sensie, że nawet, gdy przeciwnik wie, jak gramy to nie będzie w stanie poprawić swojego wyniku (widzieliśmy to na przykładzie gry w papier-kamień-nożyce). Jeżeli jednak przeciwnik jej nie zastosuje i my o tym wiemy, to znajdziemy strategię o lepszej wypłacie.

Nazwa twierdzenia bierze się stąd, że po rozpisaniu mówi ono, że
$$
  \max_\bm{x} \min_\bm{y} \bm{x}^T M \bm{y} = \min_\bm{y} \max_\bm{x} \bm{x}^T M \bm{y}.
$$
Formuła ta mówi, że nawet, gdy gracze będą wybierali (i ogłaszali) stwoje strategie po kolei, to niezależnie od kolejności przy optymalnym wyborze wypłata pozostanie ta sama. Wydaje się to intuicyjnie nieoczywiste - ale za chwilę to udowodnimy!

In [2]:
x, y = var('x y')
f = abs(x * y)
plot3d(f, (x, 0, 1), (y, 0, 1))

NameError: name 'var' is not defined

**Dowód twierdzenia o minimaksie**. Pokażemy najpierw, że pesymistyczne strategie optymalne Alicji i Boba można wyznaczyć za pomocą programowania liniowego. Potem wykażemy, że zachodzi równość $\beta(\tilde\bm{x}) = \alpha(\tilde\bm{y})$.

Zauważmy najpierw, że najlepszą odpowiedź Boba na *ustaloną* strategię $\bm{x}$ Alicji można znaleźć rozwiązując optymalizacyjny problem liniowy. 
Tzn. jeżeli $\bm{x}$ to ustalony ciąg liczb, to $\beta(\bm{x})$ to optymalna wartość następującego programu liniowego o zmiennych $y_1, y_2, \ldots y_n$:

$$
\bm{x}^T M \bm{y} \rightarrow \min
$$
$$
\sum_{j = 1}^n y_j = 1,
\bm{y} \geq 0. \quad\quad (\ast)
$$

($x_1, x_2, \ldots, x_m$ w tym programie to ustalone *parametry* i funkcja celu jest liniowa.)
Rozwiązując ten problem możemy policzyć $\beta(\bm{x})$.
Ale by znaleźć pesymistyczną strategię optymalną dla Alicji musimy zmaksymalizować $\beta$.
A funkcja $\beta(\bm{x})$ nie jest liniowa. Sytuację ratuje twierdzenie o dualności.

Konstruujemy problem dualny do (*):

$$
\bm{x}^T M \bm{y} \geq \bm{x}^T M \bm{y} - x_0 (\sum_{j=1}^n y_j - 1) = \sum_{i = 1}^m \sum_{j = 1}^n x_i m_{ij} y_j - x_0 (\sum_{j=1}^n y_j - 1) = 
$$
$$
= x_0 + \sum_{j=1}^n y_j(\sum_{i = 1}^m x_i m_{ij} - x_0) = x_0 + y^T (M^T \bm{x} - \bm{1} x_0) \geq x_0
$$


In [ ]:
# (y1, y2, ..., yn) * [M^T * x - 1 x_0]
# ta macierz po prawej to wektor kolumnowy, w j-tym jego wierszu jest wyrazenie z nawiasu po lewej stronie

Pierwsza nierówność jest tak naprawdę równością, bo wyrażenie $\sum_{j=1}^n y_j - 1$ w nawiasie jest równe zero. Kolejne przejście to zamiana z postaci macierzowej a następnie pogrupowanie po zmiennych $y_j$ ($x_0$ zostaje jako wyraz wolny - $x$-y są stałymi, parametrami). Ostatnie szacowanie jest dobre przy założeniu, że wyrażenie $M^T \bm{x} - \bm{1} x_0$ w nawiasie jest nieujemne.

Przeprowadzone szacowanie to konstrukcja problemu dualnego, którą stosowaliśmy na ćwiczeniach. Daje to problem dualny
$$
x_0 \rightarrow \max
$$
$$
M^T \bm{x} - \bm{1} x_0 \geq \bm{0}.
$$
W problemie tym jest tylko jedna zmienna $x_0$ ($\bm{x}$ jest parametrem).
Jest ona wolna: $x_0 \in \mathbb{R}$.
Z silnego twierdzenia o dualności optymalna wartość funkcji celu tego programu liniowego równa jest $\beta(\bm{x})$.

Żeby wyznaczyć $\max_\bm{x} \beta(\bm{x})$ *uzmienniamy* stałe $x_1, x_2, \ldots, x_m$ w powyższym problemie i rozwiązujemy problem
$$
x_0 \rightarrow \max
$$
$$
M^T \bm{x} - \bm{1} x_0 \geq 0,
$$
$$
\sum_{i=1}^m x_i = 1,
$$
$$
\bm{x} \geq 0, x_0 \in \mathbb{R}.
$$

Jeżeli $(\tilde x_0, \tilde\bm{x})$ oznacza optymalne rozwiązanie tego problemu liniowego, to z konstrukcji mamy
$$
\tilde x_0 = \beta(\tilde\bm{x}) = \max_\bm{x} \beta(\bm{x}).
$$

Postępując analogicznie możemy skonstruować program liniowy, którego rozwiązaniem jest optymalna strategia pesymistyczna Boba:
$$
y_0 \rightarrow \min
$$
$$
M\bm{y} - \bm{1}y_0 \leq 0,
$$
$$
\sum_{j=1}^n y_j = 1,
$$
$$
\bm{y} \geq 0, y_0 \in \mathbb{R}.
$$

Jeżeli $(\tilde y_0, \tilde\bm{y})$ jest rozwiązaniem optymalnym tego problemu, to
$$
\tilde y_0 = \alpha(\tilde\bm{y}) = \min_{\bm{y}} \alpha(\bm{y}).
$$

Znalezione tak rozwiązania $\tilde\bm{x}$, $\tilde\bm{y}$ są pesymistycznymi strategiami optymalnymi.

Dowód kończy obserwacja, że te programy liniowe są do siebie dualne (proszę doprowadzić je do postaci równościowej maksymalizacyjnej i wypisać ich macierze; porównaj dowód silnego twierdzenia o dualności).
Z silnego twierdzenia o dualności $\tilde x_0 = \tilde y_0$,
więc $\beta(\tilde\bm{x}) = \alpha(\tilde\bm{y})$ i $(\tilde\bm{x}, \tilde\bm{y})$ to równowaga Nasha, z udowodnionego wcześniej lematu.

$$
\begin{array}{c|ccccc}
 & (0, 0, 5) & (0, 1, 4) & (0, 2, 3) & (1, 1, 3) & (1, 2, 2) \\
 \hline
 (0, 0, 5) & 0 & -\frac 13 & - \frac 13 & -1 & -1 \\
 (0, 1, 4) & \frac 13 & 0 & 0 & -\frac 13 & -\frac 23 \\
 (0, 2, 3) & \frac 13 & 0 & 0 & 0 & \frac 13 \\
 (1, 1, 3) & 1 & \frac 13 & 0 & 0 & -\frac 13 \\
 (1, 2, 2) & 1 & \frac 23 & -\frac 13 & \frac 13 & 0 \\
\end{array}
$$

$$
x_0 \rightarrow \max
$$
$$
M^T \bm{x} - \bm{1} x_0 \geq 0,
$$
$$
\sum_{i=1}^m x_i = 1,
$$
$$
\bm{x} \geq 0, x_0 \in \mathbb{R}.
$$


In [4]:
M = matrix(QQ, [[0, -1/3, -1/3, -1, -1], [1/3, 0, 0, -1/3, -2/3], [1/3, 0, 0, 0, 1/3], [1, 1/3, 0, 0, -1/3], [1, 2/3, -1/3, 1/3, 0]])
show(M)

[   0 -1/3 -1/3   -1   -1]
[ 1/3    0    0 -1/3 -2/3]
[ 1/3    0    0    0  1/3]
[   1  1/3    0    0 -1/3]
[   1  2/3 -1/3  1/3    0]

In [10]:
A = transpose(M).augment(vector(QQ, [-1,-1,-1,-1,-1])).stack(vector(QQ, [1,1,1,1,1,0]))
show(A)

[   0  1/3  1/3    1    1   -1]
[-1/3    0    0  1/3  2/3   -1]
[-1/3    0    0    0 -1/3   -1]
[  -1 -1/3    0    0  1/3   -1]
[  -1 -2/3  1/3 -1/3    0   -1]
[   1    1    1    1    1    0]

In [12]:
b = vector(QQ, [0,0,0,0,0,1])
c = vector(QQ, [0,0,0,0,0,1])
P = InteractiveLPProblem(A, b, c, variable_type=['>=', '>=', '>=', '>=', '>=', ''], x=['x1','x2','x3','x4','x5','x0'], constraint_type=['>=', '>=', '>=', '>=', '>=', '=='])
show(P)

LP problem (use typeset mode to see details)

In [13]:
P.standard_form().run_simplex_method()

\begin{equation*}
\renewcommand{\arraystretch}{1.5} %notruncate
\begin{array}{|rcrcrcrcrcrcrcrcr|}
\hline
x_{8} \mspace{-6mu}&\mspace{-6mu} = \mspace{-6mu}&\mspace{-6mu} 0 \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} \frac{1}{3} x_{2} \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} \frac{1}{3} x_{3} \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} x_{4} \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} x_{5} \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} \mathit{x0}_{p} \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} \mathit{x0}_{n}\\
x_{9} \mspace{-6mu}&\mspace{-6mu} = \mspace{-6mu}&\mspace{-6mu} 0 \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} \frac{1}{3} x_{1} \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} \frac{1}{3} x_{4} \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} \frac{2}{3} x_{5} \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} \mathit{x0}_{p} \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} \mathit{x0}_{n}\\
x_{10} \mspace{-6mu}&\mspace{-6mu} = \mspace{-6mu}&\mspace{-6mu} 0 \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} \frac{1}{3} x_{1} \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} \frac{1}{3} x_{5} \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} \mathit{x0}_{p} \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} \mathit{x0}_{n}\\
x_{11} \mspace{-6mu}&\mspace{-6mu} = \mspace{-6mu}&\mspace{-6mu} 0 \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} x_{1} \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} \frac{1}{3} x_{2} \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} \frac{1}{3} x_{5} \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} \mathit{x0}_{p} \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} \mathit{x0}_{n}\\
x_{12} \mspace{-6mu}&\mspace{-6mu} = \mspace{-6mu}&\mspace{-6mu} 0 \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} x_{1} \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} \frac{2}{3} x_{2} \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} \frac{1}{3} x_{3} \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} \frac{1}{3} x_{4} \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} \mathit{x0}_{p} \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} \mathit{x0}_{n}\\
x_{13} \mspace{-6mu}&\mspace{-6mu} = \mspace{-6mu}&\mspace{-6mu} 1 \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} x_{1} \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} x_{2} \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} x_{3} \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} x_{4} \mspace{-6mu}&\mspace{-6mu} - \mspace{-6mu}&\mspace{-6mu} x_{5} \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu} \\
x_{14} \mspace{-6mu}&\mspace{-6mu} = \mspace{-6mu}&\mspace{-6mu} -1 \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} x_{1} \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} x_{2} \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} x_{3} \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} x_{4} \mspace{-6mu}&\mspace{-6mu} + \mspace{-6mu}&\mspace{-6mu} x_{5} \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu}&\mspace{-6mu} \\
\hline
z \mspace{-6mu}&\mspace{-6mu} = \mspace{-6mu}&\mspace{-6mu} 0 \mspace{-6mu}&\mspace{-6mu}  \mspace{-6mu

In [ ]:
# Strategia optymalna dla pułkownika Blotto: (0, 0, 1/2, 1/2, 0)